##### Imports

In [1]:
import sys
from pathlib import Path
import warnings
from torch.utils.data import Dataset

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import pickle
from pathlib import Path
from scr.dataset import *
# from scr.game import *
from scr.feature_engineering import *
# from scr.plot_utils import *
import gc
from scr.utils import print_scenarios

from pathlib import Path
import random
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset

from scr.utils import read_words, save_words_to_file

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words

set_seed(42)

import torch
import torch.nn as nn
from pathlib import Path
import random

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Read and Shuffle Word List
word_list = read_words('data/words_250000_train.txt') # , limit=10000)
# word_list = read_words('data/250k.txt', limit=10000)

random.shuffle(word_list)

# Calculate Frequencies and Max Word Length
word_frequencies = calculate_word_frequencies(word_list)
char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)

##### Data Path

In [2]:
import shutil
from pathlib import Path

NUM_STRATIFIED_SAMPLES = 1_00 # This will be overwritten by Papermill

# Define the base directory and the paths for training and validation parquet files
base_dataset_dir = Path("/media/sayem/510B93E12554BBD1/dataset/")

stratified_samples_dir = base_dataset_dir / str(NUM_STRATIFIED_SAMPLES)

parquet_path = stratified_samples_dir / 'parquets'
# parquet_valid_path = stratified_samples_dir / 'valid_parquets'

# Function to delete and recreate a directory
def recreate_directory(path):
    if path.exists():
        shutil.rmtree(path)  # Delete the directory and its contents
    path.mkdir(parents=True)  # Create the directory

# Recreate the train and valid directories
recreate_directory(parquet_path)
# recreate_directory(parquet_valid_path)

# print(f"Directories '{parquet_train_path}' and '{parquet_valid_path}' have been recreated.")

print(f"Directories '{parquet_path}' have been recreated.")

Directories '/media/sayem/510B93E12554BBD1/dataset/100/parquets' have been recreated.


In [3]:
len(word_list)

227300

##### Testing Data

In [4]:
# Define the total number of words and the number of test samples
from scr.custom_sampler import *
NUM_TEST_SAMPLES = 10_000

# Assuming 'word_list' contains the 250,000 words
# First, separate 10,000 words for the final testing set
testing_words = stratified_sample_by_length_and_uniqueness(
    word_list, 
    NUM_TEST_SAMPLES
)

# Define the file path for saving the testing words
testing_words_file_path = stratified_samples_dir / "testing_words.txt"

# Save the testing words to a file
with open(testing_words_file_path, 'w') as file:
    for word in testing_words:
        file.write(word + '\n')

print(f"Testing words saved in {testing_words_file_path}")

# Now, remove these testing samples from the original word list
remaining_words = [word for word in word_list if word not in testing_words]

Testing words saved in /media/sayem/510B93E12554BBD1/dataset/100/testing_words.txt


##### Stratified Sample Generation

In [5]:
## we are taking starified samples from train_words

from scr.custom_sampler import \
    stratified_sample_by_length_and_frequency, \
        stratified_sample_by_length, stratified_sample_by_length_and_uniqueness

print(f'Number of Statrified samples: {NUM_STRATIFIED_SAMPLES}')

# sampled_words_by_length_and_frequency \
#     = stratified_sample_by_length_and_frequency(train_words, \
#     word_frequencies, \
#     NUM_STRATIFIED_SAMPLES)

sampled_words_by_length = stratified_sample_by_length_and_uniqueness(remaining_words, \
    NUM_STRATIFIED_SAMPLES)

print(len(sampled_words_by_length))

Number of Statrified samples: 100
207


##### Intial State Simulation Testing

In [6]:
from scr.game import *

In [7]:
# word = "mississippi"
word = "mythopoetize"
# word = "cat"

initial_states = process_word_for_six_states(word)

initial_states

{'allMasked': '____________',
 'early': '_y__________',
 'quarterRevealed': 'my__________',
 'midRevealed': 'my__o_oe___e',
 'midLateRevealed': 'myt_o_oeti_e',
 'lateRevealed': 'mytho_oetize',
 'nearEnd': 'mytho_oetize'}

In [8]:
len(initial_states)

7

In [9]:
initial_states

{'allMasked': '____________',
 'early': '_y__________',
 'quarterRevealed': 'my__________',
 'midRevealed': 'my__o_oe___e',
 'midLateRevealed': 'myt_o_oeti_e',
 'lateRevealed': 'mytho_oetize',
 'nearEnd': 'mytho_oetize'}

##### Dataset Generation: Simulation

In [10]:
word = "mississippi"

initial_states = process_word_for_six_states(word)

# print(initial_states)
# Print generated initial states
print("Generated Initial States: ", initial_states)

Generated Initial States:  {'allMasked': '___________', 'early': 'm__________', 'quarterRevealed': 'm__________', 'midRevealed': 'm_______pp_', 'midLateRevealed': 'm_ss_ss_pp_', 'lateRevealed': 'm_ss_ss_pp_', 'nearEnd': 'm_ss_ss_pp_'}


In [11]:
from scr.game import simulate_game_progress, \
    play_game_with_a_word, process_word

# Example word and initial state
# Example usage
word = "mississippi"
# word = "cat"

initial_states = process_word_for_six_states(word)

# print(initial_states)
# Print generated initial states
print("Generated Initial States:")
i = 1 
for state_name, initial_state in initial_states.items():
    # Simulate the game
    print(initial_state)
    print(f"{i}: For initial state: {initial_state}")
    won, game_progress = simulate_game_progress(
        model=None,  # Assuming model is not used in this example
        word=word, 
        initial_state=initial_state, 
        char_frequency={},  # Assuming char_frequency is not used in this example
        max_word_length=len(word), 
        device=None,  # Assuming device is not used in this example
        max_attempts=6, 
        normalize=True,
        difficulty="medium", 
        outcome_preference='win'
    )

    # Display game progress
    for step in game_progress:
        print(f"Guessed: '{step[0]}', New State: '{step[1]}', Correct: {step[2]}")

        # break

    i+=1

    # break

    # print("Game Result:", "Won" if won else "Lost")

Generated Initial States:
___________
1: For initial state: ___________
Guessed: 'd', New State: '___________', Correct: False
Guessed: 'c', New State: '___________', Correct: False
Guessed: 'u', New State: '___________', Correct: False
Guessed: 'l', New State: '___________', Correct: False
Guessed: 'p', New State: '________pp_', Correct: True
Guessed: 'y', New State: '________pp_', Correct: False
Guessed: 'i', New State: '_i__i__ippi', Correct: True
Guessed: 'm', New State: 'mi__i__ippi', Correct: True
Guessed: 'w', New State: 'mi__i__ippi', Correct: False
m__________
2: For initial state: m__________
Guessed: 'd', New State: 'm__________', Correct: False
Guessed: 'r', New State: 'm__________', Correct: False
Guessed: 'i', New State: 'mi__i__i__i', Correct: True
Guessed: 'f', New State: 'mi__i__i__i', Correct: False
Guessed: 's', New State: 'mississi__i', Correct: True
Guessed: 'p', New State: 'mississippi', Correct: True
m__________
3: For initial state: m__________
Guessed: 'g', New

##### Writing Parquet

In [12]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Assuming the function 'process_word_for_six_states' is defined elsewhere
from scr.game import simulate_game_progress, process_word_for_six_states

def process_batch_to_parquet(batch, file_path, start_game_counter):
    game_counter = start_game_counter
    data_for_parquet = []

    for game_data in batch:
        word, state_name, initial_state, difficulty, outcome, won, game_progress = game_data
        if not game_progress:
            continue

        final_state = game_progress[-1][1]
        guessed_states = [initial_state] + [state for _, state, _ in game_progress]
        guessed_letters = [letter for letter, _, _ in game_progress]

        data_for_parquet.append({
            'game_id': game_counter,
            'word': word,
            'initial_state': initial_state,
            'final_state': final_state,
            'guessed_states': ','.join(guessed_states[:-1]),
            'guessed_letters': ','.join(guessed_letters),
            'game_state': state_name,
            'difficulty': difficulty,
            'outcome': outcome,
            'word_length': len(word),
            'won': won
        })

        game_counter += 1

    df = pd.DataFrame(data_for_parquet)
    table = pa.Table.from_pandas(df)
    pq.write_to_dataset(table, root_path=file_path, compression='snappy')

    return game_counter

def generate_batch_for_word(word):
    batch = []
    initial_states = process_word_for_six_states(word)
    games_generated = 0  # Counter for games generated

    for state_name, initial_state in initial_states.items():
        for difficulty in ["easy", "medium", "hard"]:
            for outcome in ["win", "lose"]:
                won, game_progress = simulate_game_progress(
                    model=None, word=word, initial_state=initial_state,
                    char_frequency={}, max_word_length=len(word),
                    device=None, max_attempts=6, normalize=True,
                    difficulty=difficulty, outcome_preference=outcome
                )
                batch.append((word, state_name, initial_state, \
                    difficulty, outcome, won, game_progress))
                games_generated += 1  # Increment game counter

    return batch, games_generated

def main_execution(words, parquet_path):
    game_counter = 0
    games_per_word = len(initial_states) * 3 * 2  # 7 states, 3 difficulties, 2 outcomes

    for word in tqdm(words, desc="Processing Words"):
        batch, games_generated = generate_batch_for_word(word)

        # Update game count
        game_counter += games_generated

        # Process and save batch
        process_batch_to_parquet(batch, parquet_path, game_counter)

    print(f"Total games processed: {game_counter}")

In [13]:
# =======================================================
# Execute the main function
main_execution(sampled_words_by_length, parquet_path)
# =======================================================

Processing Words: 100%|██████████| 207/207 [00:00<00:00, 285.70it/s]

Total games processed: 8694


##### Checking the dataset 

In [14]:
import pandas as pd
from pathlib import Path


# Find all Parquet files in the directory
parquet_files = list(parquet_path.glob('*.parquet'))

if parquet_files:
    total_game_sequences = 0

    # Iterate over each file and sum the number of game sequences
    for file in parquet_files:
        df = pd.read_parquet(file)
        total_game_sequences += len(df)

    print(f"Total number of game sequences across all files: {total_game_sequences}")
else:
    print("No Parquet files found in the specified directory.")


Total number of game sequences across all files: 8658


In [15]:
import pandas as pd
import glob
from pathlib import Path

# Use glob to find all Parquet files in the folder
parquet_files = parquet_path.glob('*.parquet')

# Read and concatenate all Parquet files into a single DataFrame
df = pd.concat([pd.read_parquet(file) for file in \
    parquet_files], ignore_index=True)

# # Display the first few rows of the DataFrame
# print(df.head())

# Get the total number of rows (games) in the DataFrame
total_games = len(df)
print(f"Total number of games in the dataset: {total_games}")

# Additional checks and summary statistics
print("Null values in each column:")
print(df.isnull().sum())

print("\nSummary statistics:")
print(df.describe())

# Count the number of unique words or game states
unique_words = df['word'].nunique()
print(f"\nNumber of unique words: {unique_words}")

# Inspect the distribution of game outcomes, difficulties, etc.
print("\nOutcome distribution:")
print(df['outcome'].value_counts())

print("\nDifficulty distribution:")
print(df['difficulty'].value_counts())

# Word length distribution
print("\nWord Length Distribution:")
print(df['word_length'].value_counts())

Total number of games in the dataset: 8658
Null values in each column:
game_id            0
word               0
initial_state      0
final_state        0
guessed_states     0
guessed_letters    0
game_state         0
difficulty         0
outcome            0
word_length        0
won                0
dtype: int64

Summary statistics:
           game_id  word_length
count  8658.000000  8658.000000
mean   4375.514553    12.502426
std    2506.980863     5.490101
min      42.000000     1.000000
25%    2206.250000     8.000000
50%    4370.500000    12.000000
75%    6534.750000    17.000000
max    8735.000000    29.000000

Number of unique words: 207

Outcome distribution:
outcome
win     4329
lose    4329
Name: count, dtype: int64

Difficulty distribution:
difficulty
easy      2886
medium    2886
hard      2886
Name: count, dtype: int64

Word Length Distribution:
word_length
9     714
8     672
10    630
12    546
7     546
11    504
13    462
6     462
14    420
15    378
18    378
17    3

In [16]:
import pandas as pd

# # Replace this with the path to your Parquet file
# parquet_file_path = 'path/to/your/HangmanData.parquet'

# Read the Parquet file
df = pd.read_parquet(parquet_path)

# Display the first few rows of the DataFrame
df

,game_id,word,initial_state,final_state,guessed_states,guessed_letters,game_state,difficulty,outcome,word_length,won
0,6930,overconscientiousness,_____________________,overconscientiousness,"_____________________,_____________________,__...","f,u,d,v,t,o,c,r,e,i,q,s,z,n",allMasked,easy,win,21,True
1,6931,overconscientiousness,_____________________,overconscientiousness,"_____________________,_____________________,__...","l,r,x,c,n,u,w,v,y,i,s,e,o,t",allMasked,easy,lose,21,True
2,6932,overconscientiousness,_____________________,o__r_o_s______ous__ss,"_____________________,_____________________,__...","h,u,z,a,r,b,s,g,o,x",allMasked,medium,win,21,False
3,6933,overconscientiousness,_____________________,ov_r_o___i___io______,"_____________________,_v___________________,_v...","v,j,f,z,o,r,k,i,p,d",allMasked,medium,lose,21,False
4,6934,overconscientiousness,_____________________,____c___c____________,"_____________________,____c___c____________,__...","c,a,j,z,q,k,x",allMasked,hard,win,21,False
...,...,...,...,...,...,...,...,...,...,...,...
8653,499,discarded,_iscar_e_,discarded,_iscar_e_,d,nearEnd,easy,lose,9,True
8654,500,discarded,_iscar_e_,discarded,_iscar_e_,d,nearEnd,medium,win,9,True
8655,501,discarded,_iscar_e_,discarded,"_iscar_e_,_iscar_e_,_iscar_e_","v,u,d",nearEnd,medium,lose,9,True
8656,502,discarded,_iscar_e_,_iscar_e_,"_iscar_e_,_iscar_e_,_iscar_e_,_iscar_e_,_iscar...","k,p,x,o,l,v",nearEnd,hard,win,9,False


In [17]:
# Get the total number of rows (games) in the DataFrame
total_games = len(df)

print(f"Total number of games in the dataset: {total_games}")

# Additional checks you might want to perform:
# - Check for any null values or anomalies in the data
print(df.isnull().sum())

# - Get a summary of the DataFrame
print(df.describe())

# - Count the number of unique words or game states
unique_words = df['word'].nunique()
print(f"Number of unique words: {unique_words}")

# - Inspect the distribution of game outcomes, difficulties, etc.
print(df['outcome'].value_counts())
print(df['difficulty'].value_counts())

Total number of games in the dataset: 8658
game_id            0
word               0
initial_state      0
final_state        0
guessed_states     0
guessed_letters    0
game_state         0
difficulty         0
outcome            0
word_length        0
won                0
dtype: int64
           game_id  word_length
count  8658.000000  8658.000000
mean   4375.514553    12.502426
std    2506.980863     5.490101
min      42.000000     1.000000
25%    2206.250000     8.000000
50%    4370.500000    12.000000
75%    6534.750000    17.000000
max    8735.000000    29.000000
Number of unique words: 207
outcome
win     4329
lose    4329
Name: count, dtype: int64
difficulty
easy      2886
medium    2886
hard      2886
Name: count, dtype: int64


##### Dataset Creation and train test split

In [1]:
# Create datasets directly from the saved parquet files
hangman_dataset = HangmanDataset(parquet_path)
# valid_dataset = HangmanDataset(parquet_valid_path)

NameError: name 'HangmanDataset' is not defined

In [19]:
hangman_dataset[(29,)]

{'game_id': 1407,
 'word': 'maladapted',
 'initial_state': ['_a_a_a_te_'],
 'final_state': 'maladapted',
 'guessed_states': ['_a_a_a_te_',
  'ma_a_a_te_',
  'ma_a_a_te_',
  'ma_a_apte_',
  'ma_a_apte_',
  'mala_apte_'],
 'guessed_letters': ['m', 'u', 'p', 'q', 'l', 'd'],
 'game_state': 'midRevealed',
 'difficulty': 'medium',
 'outcome': 'lose',
 'word_length': 10,
 'won': False}

In [20]:
hangman_dataset.get_group_info()

{('easy', 'win'): 1443,
 ('easy', 'lose'): 1443,
 ('medium', 'win'): 1443,
 ('medium', 'lose'): 1443,
 ('hard', 'win'): 1443,
 ('hard', 'lose'): 1443}